# Agents Testing Notebook

This notebook tests the HR Agent and agent workflows.

## 🤖 What We'll Test
- Agent creation using factory
- Agent with memory (conversation persistence)
- Multi-turn conversations
- Tool calling workflows
- Error handling
- Complex HR workflows

**Last Updated**: January 2025

## Setup

In [1]:
import sys
import os

# Add project root to path
project_root = os.path.abspath('../..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
os.chdir(project_root)

print(f"✅ Project root: {project_root}")
print(f"✅ Current directory: {os.getcwd()}")

# Load configuration
from config import settings
print(f"\n✅ Configuration loaded")
print(f"   Tool Mode: {settings.TOOL_MODE}")
print(f"   Model: {settings.MODEL_NAME}")
print(f"   Temperature: {settings.TEMPERATURE}")

✅ Project root: /home/shamaseen/Desktop/Projects/personal/Langchain/tutorial/whatsapp_hr_assistant
✅ Current directory: /home/shamaseen/Desktop/Projects/personal/Langchain/tutorial/whatsapp_hr_assistant

✅ Configuration loaded
   Tool Mode: mcp
   Model: gemini-2.5-flash
   Temperature: 0.7


## Test 1: Create Agent Using Factory

Test creating the HR agent using the factory function

In [2]:
from agents.hr_agent import create_agent
from langchain_core.messages import HumanMessage

print("Creating HR Agent...\n" + "="*60)

# Create agent
agent = create_agent()
print("✅ Agent created successfully")
print("   Agent type:", type(agent).__name__)
print("   Has checkpointer:", hasattr(agent, 'checkpointer'))

print("\n" + "="*60)
print("✅ Agent factory test complete!")

Creating HR Agent...
✅ Loaded saved OAuth token
✅ Google Services initialized with OAuth 2.0
   ✓ Registered tool: cv_sheet_manager
   ✓ Registered tool: gmail
   ✓ Registered tool: calendar
   ✓ Registered tool: webex
   ✓ Registered tool: datetime
   ✓ Registered tool: process_cvs
   ✓ Registered tool: search_candidates
   ✓ Registered tool: search_create_sheet
✅ Loaded MCP tools: 8 tools registered
   Tool names: cv_sheet_manager, gmail, calendar, webex, datetime, process_cvs, search_candidates, search_create_sheet
✅ Agent configured with 1 tools
✅ LangGraph PostgreSQL checkpointer tables initialized
✅ Checkpointer ready with autocommit=True
✅ Agent created successfully
   Agent type: CompiledStateGraph
   Has checkpointer: True

✅ Agent factory test complete!


## Test 2: Simple Query (No Tools)

Test agent with a simple query that doesn't require tools

In [3]:
from agents.hr_agent import create_agent
from langchain_core.messages import HumanMessage

print("Testing Simple Query...\n" + "="*60)

agent = create_agent()

# IMPORTANT: Must provide config with thread_id for checkpointer
config = {"configurable": {"thread_id": "test_simple_123"}}

query = "Hello! Can you tell me what you can help me with?"
print(f"Query: {query}\n")

result = agent.invoke({
    "messages": [HumanMessage(content=query)],
    "sender_phone": "test_simple_123",
    "sender_identifier": "test@example.com"
}, config=config)

print("Response:")
print("-" * 60)
print(result['messages'][-1].content)
print("-" * 60)

print(f"\n✅ Conversation had {len(result['messages'])} message(s)")

print("\n" + "="*60)
print("✅ Simple query test complete!")

Testing Simple Query...
   ✓ Registered tool: cv_sheet_manager
   ✓ Registered tool: gmail
   ✓ Registered tool: calendar
   ✓ Registered tool: webex
   ✓ Registered tool: datetime
   ✓ Registered tool: process_cvs
   ✓ Registered tool: search_candidates
   ✓ Registered tool: search_create_sheet
✅ Loaded MCP tools: 8 tools registered
   Tool names: cv_sheet_manager, gmail, calendar, webex, datetime, process_cvs, search_candidates, search_create_sheet
✅ Agent configured with 1 tools
Query: Hello! Can you tell me what you can help me with?

Response:
------------------------------------------------------------
I can assist you with a variety of HR recruitment tasks! Here's a summary of what I can do:

*   **CV Management:** Process CVs from Google Drive, extract data, create/manage candidate sheets, and search/rank candidates for specific job positions.
*   **Communication:** Send, receive, read, reply to, and search emails using Gmail.
*   **Scheduling:** Create, list, get details for, 

## Test 3: Query With Tool Call

Test agent with a query that requires using a tool (datetime)

In [4]:
from agents.hr_agent import create_agent
from langchain_core.messages import HumanMessage

print("Testing Query With Tool Call...\n" + "="*60)

agent = create_agent()

# IMPORTANT: Must provide config with thread_id
config = {"configurable": {"thread_id": "test_tool_456"}}

query = "What is the current date and time?"
print(f"Query: {query}\n")

result = agent.invoke({
    "messages": [HumanMessage(content=query)],
    "sender_phone": "test_tool_456",
    "sender_identifier": "test@example.com"
}, config=config)

print("\nConversation Flow:")
print("-" * 60)
for i, msg in enumerate(result['messages'], 1):
    msg_type = type(msg).__name__
    if hasattr(msg, 'content'):
        content = msg.content[:100] if msg.content else "[No content]"
    else:
        content = str(msg)[:100]
    print(f"{i}. [{msg_type}] {content}")

print("-" * 60)

print("\nFinal Response:")
print("-" * 60)
print(result['messages'][-1].content)
print("-" * 60)

print(f"\n✅ Total messages: {len(result['messages'])}")

print("\n" + "="*60)
print("✅ Tool call test complete!")

Testing Query With Tool Call...
   ✓ Registered tool: cv_sheet_manager
   ✓ Registered tool: gmail
   ✓ Registered tool: calendar
   ✓ Registered tool: webex
   ✓ Registered tool: datetime
   ✓ Registered tool: process_cvs
   ✓ Registered tool: search_candidates
   ✓ Registered tool: search_create_sheet
✅ Loaded MCP tools: 8 tools registered
   Tool names: cv_sheet_manager, gmail, calendar, webex, datetime, process_cvs, search_candidates, search_create_sheet
✅ Agent configured with 1 tools
Query: What is the current date and time?


Conversation Flow:
------------------------------------------------------------
1. [HumanMessage] What is the current date and time?
2. [SystemMessage] You are a helpful and reliable HR recruitment assistant with access to multiple tools for CV process
3. [AIMessage] [No content]
4. [ToolMessage] {"success": true, "current_datetime": "2025-10-25T10:26:51.090496+00:00", "date": "2025-10-25", "tim
5. [AIMessage] The current date and time is Saturday, October 

## Test 4: Memory Persistence

Test that the agent remembers information across multiple interactions

In [5]:
from agents.hr_agent import create_agent
from langchain_core.messages import HumanMessage

print("Testing Memory Persistence...\n" + "="*60)

agent = create_agent()

# Use same thread_id for both messages to test memory
thread_id = "test_memory_789"
config = {"configurable": {"thread_id": thread_id}}

# First interaction - provide information
print("\n--- First Interaction ---")
query1 = "My name is Alice and I'm applying for the Senior Developer position."
print(f"Query: {query1}\n")

result1 = agent.invoke({
    "messages": [HumanMessage(content=query1)],
    "sender_phone": thread_id,
    "sender_identifier": f"{thread_id}@example.com"
}, config=config)

print("Response:")
print(result1['messages'][-1].content)

# Second interaction - test if agent remembers
print("\n--- Second Interaction ---")
query2 = "What position did I say I was applying for?"
print(f"Query: {query2}\n")

result2 = agent.invoke({
    "messages": [HumanMessage(content=query2)],
    "sender_phone": thread_id,
    "sender_identifier": f"{thread_id}@example.com"
}, config=config)

print("Response:")
print(result2['messages'][-1].content)

# Third interaction - test name recall
print("\n--- Third Interaction ---")
query3 = "What's my name?"
print(f"Query: {query3}\n")

result3 = agent.invoke({
    "messages": [HumanMessage(content=query3)],
    "sender_phone": thread_id,
    "sender_identifier": f"{thread_id}@example.com"
}, config=config)

print("Response:")
print(result3['messages'][-1].content)

print("\n" + "="*60)
print("✅ Memory persistence test complete!")
print("   The agent should remember both the name (Alice) and position (Senior Developer)")

Testing Memory Persistence...
   ✓ Registered tool: cv_sheet_manager
   ✓ Registered tool: gmail
   ✓ Registered tool: calendar
   ✓ Registered tool: webex
   ✓ Registered tool: datetime
   ✓ Registered tool: process_cvs
   ✓ Registered tool: search_candidates
   ✓ Registered tool: search_create_sheet
✅ Loaded MCP tools: 8 tools registered
   Tool names: cv_sheet_manager, gmail, calendar, webex, datetime, process_cvs, search_candidates, search_create_sheet
✅ Agent configured with 1 tools

--- First Interaction ---
Query: My name is Alice and I'm applying for the Senior Developer position.

Response:
It's great to hear from you, Alice! While I appreciate you letting me know about your application, as an AI assistant, I don't directly handle job applications or personal information in this chat.

To apply for the Senior Developer position, please make sure you've submitted your CV through our official application portal or to the designated HR email address.

If you've already submitted 

## Test 5: Complex HR Workflow

Test a complete HR workflow with multiple tool calls

In [6]:
from agents.hr_agent import create_agent
from langchain_core.messages import HumanMessage

print("Testing Complex HR Workflow...\n" + "="*60)

agent = create_agent()
config = {"configurable": {"thread_id": "test_complex_999"}}

# Complex query that might trigger multiple tools
query = """
I need help with the following:
1. Check what time it is now
2. List all candidates in our CV database
3. Schedule a meeting for tomorrow
"""

print(f"Query: {query}\n")

result = agent.invoke({
    "messages": [HumanMessage(content=query)],
    "sender_phone": "test_complex_999",
    "sender_identifier": "hr@example.com"
}, config=config)

print("\nConversation Summary:")
print("-" * 60)
print(f"Total messages: {len(result['messages'])}")

# Count tool calls
tool_calls = 0
for msg in result['messages']:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        tool_calls += len(msg.tool_calls)

print(f"Tool calls made: {tool_calls}")

print("\nFinal Response:")
print("-" * 60)
print(result['messages'][-1].content)
print("-" * 60)

print("\n" + "="*60)
print("✅ Complex workflow test complete!")

Testing Complex HR Workflow...
   ✓ Registered tool: cv_sheet_manager
   ✓ Registered tool: gmail
   ✓ Registered tool: calendar
   ✓ Registered tool: webex
   ✓ Registered tool: datetime
   ✓ Registered tool: process_cvs
   ✓ Registered tool: search_candidates
   ✓ Registered tool: search_create_sheet
✅ Loaded MCP tools: 8 tools registered
   Tool names: cv_sheet_manager, gmail, calendar, webex, datetime, process_cvs, search_candidates, search_create_sheet
✅ Agent configured with 1 tools
Query: 
I need help with the following:
1. Check what time it is now
2. List all candidates in our CV database
3. Schedule a meeting for tomorrow



Conversation Summary:
------------------------------------------------------------
Total messages: 7
Tool calls made: 2

Final Response:
------------------------------------------------------------
It is currently 10:27:07 UTC on Saturday, October 25, 2025.

I encountered an error when trying to list all candidates. The tool requires a "sheet_id" and "job

## Test 6: Error Handling

Test how the agent handles errors and invalid requests

In [7]:
from agents.hr_agent import create_agent
from langchain_core.messages import HumanMessage

print("Testing Error Handling...\n" + "="*60)

agent = create_agent()
config = {"configurable": {"thread_id": "test_error_111"}}

# Test 1: Impossible request
print("\nTest 1: Impossible request")
query1 = "Please schedule a meeting in the past (yesterday at 2 PM)"
print(f"Query: {query1}\n")

result1 = agent.invoke({
    "messages": [HumanMessage(content=query1)],
    "sender_phone": "test_error_111",
    "sender_identifier": "test@example.com"
}, config=config)

print("Response:")
print(result1['messages'][-1].content[:200])

# Test 2: Ambiguous request
print("\n\nTest 2: Ambiguous request")
query2 = "Send an email"
print(f"Query: {query2}\n")

result2 = agent.invoke({
    "messages": [HumanMessage(content=query2)],
    "sender_phone": "test_error_111",
    "sender_identifier": "test@example.com"
}, config=config)

print("Response:")
print(result2['messages'][-1].content[:200])

print("\n" + "="*60)
print("✅ Error handling test complete!")
print("   The agent should handle errors gracefully and ask for clarification")

Testing Error Handling...
   ✓ Registered tool: cv_sheet_manager
   ✓ Registered tool: gmail
   ✓ Registered tool: calendar
   ✓ Registered tool: webex
   ✓ Registered tool: datetime
   ✓ Registered tool: process_cvs
   ✓ Registered tool: search_candidates
   ✓ Registered tool: search_create_sheet
✅ Loaded MCP tools: 8 tools registered
   Tool names: cv_sheet_manager, gmail, calendar, webex, datetime, process_cvs, search_candidates, search_create_sheet
✅ Agent configured with 1 tools

Test 1: Impossible request
Query: Please schedule a meeting in the past (yesterday at 2 PM)

Response:
I cannot schedule a meeting in the past. Please provide a future date and time for the meeting.


Test 2: Ambiguous request
Query: Send an email

Response:
I can help with that. What is the subject and body of the email? To whom should it be sent?

✅ Error handling test complete!
   The agent should handle errors gracefully and ask for clarification


## Test 7: Real HR Scenario

Simulate a realistic HR assistant conversation

In [10]:
from agents.hr_agent import create_agent
from langchain_core.messages import HumanMessage

print("Testing Real HR Scenario...\n" + "="*60)

agent = create_agent()
thread_id = "hr_scenario_001"
config = {"configurable": {"thread_id": thread_id}}

# Simulate a multi-turn HR conversation
conversation = [
    "Hi, I'm interested in the Senior Developer position.",
    "My name is Bob Smith and my email is bob.smith@email.com",
    "I have 7 years of experience in Python and AI development",
    "Can you check if there are any upcoming interview slots available?",
]

for i, query in enumerate(conversation, 1):
    print(f"\n--- Turn {i} ---")
    print(f"Candidate: {query}\n")
    
    result = agent.invoke({
        "messages": [HumanMessage(content=query)],
        "sender_phone": thread_id,
        "sender_identifier": "bob.smith@email.com"
    }, config=config)
    
    print(f"HR Assistant: {result['messages'][-1].content}")
    print("-" * 60)

print("\n" + "="*60)
print("✅ Real HR scenario test complete!")
print("   The agent maintained context throughout the conversation")

Testing Real HR Scenario...
   ✓ Registered tool: cv_sheet_manager
   ✓ Registered tool: gmail
   ✓ Registered tool: calendar
   ✓ Registered tool: webex
   ✓ Registered tool: datetime
   ✓ Registered tool: process_cvs
   ✓ Registered tool: search_candidates
   ✓ Registered tool: search_create_sheet
✅ Loaded MCP tools: 8 tools registered
   Tool names: cv_sheet_manager, gmail, calendar, webex, datetime, process_cvs, search_candidates, search_create_sheet
✅ Agent configured with 1 tools

--- Turn 1 ---
Candidate: Hi, I'm interested in the Senior Developer position.

HR Assistant: Great! To help you with the Senior Developer position, could you please tell me if you've already submitted your CV?

If not, I can help you process your CV and add you to our candidate pool. Just let me know!
------------------------------------------------------------

--- Turn 2 ---
Candidate: My name is Bob Smith and my email is bob.smith@email.com

HR Assistant: Thanks, Bob. To process your application for

## Test Summary

In [9]:
print("\n" + "="*80)
print("📊 AGENT TESTING SUMMARY")
print("="*80)
print("")

print("✅ Tested Scenarios:")
print("  1. Agent Factory - Creating agents using factory function")
print("  2. Simple Query - Agent responses without tools")
print("  3. Tool Calling - Agent using datetime and other tools")
print("  4. Memory Persistence - Conversation context across turns")
print("  5. Complex Workflows - Multi-tool HR workflows")
print("  6. Error Handling - Graceful handling of errors")
print("  7. Real HR Scenario - Complete multi-turn conversation")
print("")

print("🔑 Key Findings:")
print("  • Agent requires config with thread_id for checkpointer")
print("  • Memory persists across multiple turns")
print("  • Tools are called automatically when needed")
print("  • Error handling is graceful and informative")
print("")

print("="*80)
print("🎉 ALL AGENT TESTS COMPLETED!")
print("="*80)
print("")
print("Next Steps:")
print("1. Build custom agent: See 03_custom_agent_tutorial.ipynb")
print("2. Test MCP integration: See 04_mcp_integration.ipynb")
print("="*80)


📊 AGENT TESTING SUMMARY

✅ Tested Scenarios:
  1. Agent Factory - Creating agents using factory function
  2. Simple Query - Agent responses without tools
  3. Tool Calling - Agent using datetime and other tools
  4. Memory Persistence - Conversation context across turns
  5. Complex Workflows - Multi-tool HR workflows
  6. Error Handling - Graceful handling of errors
  7. Real HR Scenario - Complete multi-turn conversation

🔑 Key Findings:
  • Agent requires config with thread_id for checkpointer
  • Memory persists across multiple turns
  • Tools are called automatically when needed
  • Error handling is graceful and informative

🎉 ALL AGENT TESTS COMPLETED!

Next Steps:
1. Build custom agent: See 03_custom_agent_tutorial.ipynb
2. Test MCP integration: See 04_mcp_integration.ipynb
